### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

#### Attempt 1
Initially, I set out to have a simple star schema with one fact table for Covid-19 cases and deaths over time, with a references to counties and states. This could have worked since I could have treated states as a special kind of county (states technically have a FIPS code as well, which is just the 2-digit state code followed by three zeroes). The county table would be a dimension table that is reference from the Covid-19 fact table.

I decided against this for the following reasons:
* I want counties to refer to the states they're in. If states are counties as well, it gets weird quickly because I'd need to have counties have a foreign key to... another county (which is just a state in a trenchcoat). This just opens us up to data corruption because we'll somehow end up with a county referring to something that isn't a state.
* It's not intuitive. I wouldn't expect to see states in a table for counties.
* Some of my data sources (weather, and optionally election and mobility) don't have data for states, so I'd have to awkwardly infer it somehow, or leave those columns empty. Again, this assumes that the end user understands that the county data contains "special counties", i.e. states.

#### Attempt 2
Alright, we'll have two dimension tables, then: One for counties, and one for states. So how does this affect the facts table? We'd now need each row to refer to both a county and a state, but only one of these would ever be set at a time, the other being "None", since we can't have a row that has data for both a state and a county. That's still weird, I don't want to have None-values without good reasons, and I don't want end users to have to remember this when dealing with my data.

Additionally, some of my supplementary data sources are time series as well! Weather data is recorded by date, and so is mobility data. But these aren't available for states, and generating averages for states based on county data feels icky (e.g. California is a huge state, I'd expect the weather data to differ a lot between the North and South ends, an average wouldn't be very useful).  
But wouldn't it be great to record those time series in the same fact table, since they also use the county and the date? So in addition to Covid-19 data, this time series fact table could contain data about the day-to-day weather (and, later on, resident mobility)!

#### Attempt 3
Since this data isn't avaiable for states, maybe let's split the facts table in two: One for a county time series, and one for a state time series. We can augment the county facts table with weather and mobility data, but leave the states only using Covid-19 data.

Now, as an alternative, we could not create a facts table for states and instead generate aggregates for case and death numbers based on counties, since they refer to states. The downside would be that if we wanted to query case numbers and combine it with information about the state, we'd need to join the facts table with counties and states. This kind of query seems common enough to me that I wouldn't want to require users to constantly run groupBy-aggregates and a three-way join; instead, we can just bake that data into another facts table once and then access it directly.  
I'm retaining the reference to the state from the county table. I'd expect the reference be used for grouping by state or selecting only counties in a specific state, so generally we wouldn't need to join with the state dimension table. I can however see some cases where it would be advantageous to join with the state table; for example, if we want to find counties that are doing better/worse in some metric than the state average (averaging grouped counties in a state won't work in this case, since counties have different populations, so statistics based on percentages of popuplation will be distorted by averaging indiscriminately).

#### Conclusion
All in all, this leaves us with the following schema:

<img src="images/database_schema.png">

(This database schema visualisation was created using [SQLDBM](app.sqldbm.com))

#### Columns
County IDs are FIPS codes which are 5 digits, hence the character limit. State IDs are state name abbreviations which are always two letters. I hope that 50 characters are enough for county and state names, but I'll adjust this if I run into trouble later on.  
The county and state dimension tables allow NULL values for some columns. These are the columns that I identified to have NaNs during exploratory analysis. I'd rather require end users make a check to skip those rows than to generate questionable replacement data and lead them to draw incorrect conclusions, especially if it is clearly telegraphed in the table configuration that these columns will have NULL-values.  
Also note how the state dimension table has a lot fewer columns with NULL-values.

The fact tables for county and state time series contain columns for Covid-19 case and death counts, both as a total and as a delta to the previous day. We could do with either of these and then calculate the other one based on it, but I predict that both versions will be in common use, so I'd rather build them in instead of repeatedly calculating them for each request later on.

The date table uses a timestamp as a primary key, and at table population I will fill in the timestamp components instead of having to extract them constantly at execution time.

#### Table creation
The resulting table creation (for Redshift) can be found in the "sql" folder, in the "create_tables.sql" file.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

##### Outline
Pipeline:
- Only use counties and states that are in the Covid and health data sets
- Use spark, mostly rely on dataframes but sometimes use temporary database views for convenience
- End result should be outputting parquet files

County facts:
- Start with the Covid-19 data set, use the timestamps to link to time, use FIPS to link to counties
- Add total covid cases and deaths from the data set
- Calculate delta based on current and previous columns during insert so we don't have to calculate the previous date and extract that data later
- Augment with weather data based on the timestamp and county
- Partition by county, then month

State dim:
- Create state dimension table from health data as a start
- Calculate area and population density based on county dim table, group by state
- Don't partition? There's only 50 of them anyway

State facts:
- Start Covid-19 data, use the state abbreviation as a link to states, timestamp to time
- Calculate aggregate Covid-19 cases and deaths (both total and delta) for each day based on county facts table, group by state
- Partition by state, then month

##### Setup
I'm going to need Spark for this because I'll want to make use of some of its functionality, such as the ability to create temporary SQL views of my dataframes.

In [1]:
from setup import create_spark_session

spark = create_spark_session()

In [13]:
import pandas as pd
from pyspark.sql.functions import col, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, from_unixtime
from pyspark.sql.types import *

from scripts.clean import *

In [3]:
# For now, just locally, later on maybe write this to S3 instead
output_path = "output/"

##### Time dimension table
- Create using pandas, start at 1st of January 2020, go in one-day intervals until 31st of December 2020 and create a timestamp for each day
- Extract time components
- Partition by month (better would be year, then month, but we're only doing 2020)

In [4]:
time_df_pd = pd.DataFrame({'date':pd.date_range('2020-01-01', '2020-12-31')})
time_df_pd.head()

,date
0,2020-01-01
1,2020-01-02
2,2020-01-03
3,2020-01-04
4,2020-01-05


In [5]:
time_df_pd.count()

date    366
dtype: int64

In [6]:
time_df = spark.createDataFrame(time_df_pd)
time_df.show()

+-------------------+
|               date|
+-------------------+
|2020-01-01 00:00:00|
|2020-01-02 00:00:00|
|2020-01-03 00:00:00|
|2020-01-04 00:00:00|
|2020-01-05 00:00:00|
|2020-01-06 00:00:00|
|2020-01-07 00:00:00|
|2020-01-08 00:00:00|
|2020-01-09 00:00:00|
|2020-01-10 00:00:00|
|2020-01-11 00:00:00|
|2020-01-12 00:00:00|
|2020-01-13 00:00:00|
|2020-01-14 00:00:00|
|2020-01-15 00:00:00|
|2020-01-16 00:00:00|
|2020-01-17 00:00:00|
|2020-01-18 00:00:00|
|2020-01-19 00:00:00|
|2020-01-20 00:00:00|
+-------------------+
only showing top 20 rows



In [7]:
# Spark 3.0+ for some reason removed the ability to parse weekdays into integers, it only supports strings now.
# Don't ask me why, I can't see how that's a good restriction to add.
# We can fall back to the legacy time parser to restore the old behaviour.
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [8]:
time_df = time_df.withColumn('day', dayofmonth('date')) \
    .withColumn('week', weekofyear('date')) \
    .withColumn('month', month('date')) \
    .withColumn('year', year('date')) \
    .withColumn('weekday', date_format(col("date"), "u"))
time_df.show()

+-------------------+---+----+-----+----+-------+
|               date|day|week|month|year|weekday|
+-------------------+---+----+-----+----+-------+
|2020-01-01 00:00:00|  1|   1|    1|2020|      3|
|2020-01-02 00:00:00|  2|   1|    1|2020|      4|
|2020-01-03 00:00:00|  3|   1|    1|2020|      5|
|2020-01-04 00:00:00|  4|   1|    1|2020|      6|
|2020-01-05 00:00:00|  5|   1|    1|2020|      7|
|2020-01-06 00:00:00|  6|   2|    1|2020|      1|
|2020-01-07 00:00:00|  7|   2|    1|2020|      2|
|2020-01-08 00:00:00|  8|   2|    1|2020|      3|
|2020-01-09 00:00:00|  9|   2|    1|2020|      4|
|2020-01-10 00:00:00| 10|   2|    1|2020|      5|
|2020-01-11 00:00:00| 11|   2|    1|2020|      6|
|2020-01-12 00:00:00| 12|   2|    1|2020|      7|
|2020-01-13 00:00:00| 13|   3|    1|2020|      1|
|2020-01-14 00:00:00| 14|   3|    1|2020|      2|
|2020-01-15 00:00:00| 15|   3|    1|2020|      3|
|2020-01-16 00:00:00| 16|   3|    1|2020|      4|
|2020-01-17 00:00:00| 17|   3|    1|2020|      5|


In [9]:
# Even though the original pandas dataframe used datetime, the spark dataframe reverted to timestamp.
# I really don't need the time-of-day parts, so let's force this back to datetime.
time_df = time_df.withColumn('date', time_df['date'].cast(DateType()))
time_df.show()

+----------+---+----+-----+----+-------+
|      date|day|week|month|year|weekday|
+----------+---+----+-----+----+-------+
|2020-01-01|  1|   1|    1|2020|      3|
|2020-01-02|  2|   1|    1|2020|      4|
|2020-01-03|  3|   1|    1|2020|      5|
|2020-01-04|  4|   1|    1|2020|      6|
|2020-01-05|  5|   1|    1|2020|      7|
|2020-01-06|  6|   2|    1|2020|      1|
|2020-01-07|  7|   2|    1|2020|      2|
|2020-01-08|  8|   2|    1|2020|      3|
|2020-01-09|  9|   2|    1|2020|      4|
|2020-01-10| 10|   2|    1|2020|      5|
|2020-01-11| 11|   2|    1|2020|      6|
|2020-01-12| 12|   2|    1|2020|      7|
|2020-01-13| 13|   3|    1|2020|      1|
|2020-01-14| 14|   3|    1|2020|      2|
|2020-01-15| 15|   3|    1|2020|      3|
|2020-01-16| 16|   3|    1|2020|      4|
|2020-01-17| 17|   3|    1|2020|      5|
|2020-01-18| 18|   3|    1|2020|      6|
|2020-01-19| 19|   3|    1|2020|      7|
|2020-01-20| 20|   4|    1|2020|      1|
+----------+---+----+-----+----+-------+
only showing top

In [10]:
time_df.write.partitionBy('month').mode('overwrite').parquet(output_path + "time.parquet")

##### County dimension table:
- Create county dimension table from Covid-19 data as a start
- Augment county table with health data
- Augment with area and population density (using health data and area data)
- Use state abbreviation as foreign key for state table (which is created afterwards)
- Partition by state

In [ ]:
covid_cases_df = clean_covid_data(spark, "data/covid_cases_US.csv")